In [3]:
dofile("Code/utils.lua")

cannot open Code/utils.lua: No such file or directory
stack traceback:
	/home/francisco/torch/install/share/lua/5.2/itorch/main.lua:167: in function </home/francisco/torch/install/share/lua/5.2/itorch/main.lua:160>
	[C]: in function 'dofile'
	[string "local f = function() return dofile("Code/util..."]:1: in function 'f'
	[string "local f = function() return dofile("Code/util..."]:1: in main chunk
	[C]: in function 'xpcall'
	/home/francisco/torch/install/share/lua/5.2/itorch/main.lua:210: in function </home/francisco/torch/install/share/lua/5.2/itorch/main.lua:174>
	(...tail calls...)
	/home/francisco/torch/install/share/lua/5.2/lzmq/poller.lua:75: in function 'poll'
	...francisco/torch/install/share/lua/5.2/lzmq/impl/loop.lua:307: in function 'poll'
	...francisco/torch/install/share/lua/5.2/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...francisco/torch/install/share/lua/5.2/lzmq/impl/loop.lua:370: in function 'start'
	/home/francisco/torch/install/share/lua/5.2/itorch/main.lua:389:

In [7]:
paths.dir("./")

{
  1 : ..
  2 : Simulations
  3 : Performance
  4 : .
  5 : runModel.lua
  6 : utils.lua
  7 : utilsNN.lua
  8 : .DS_Store
  9 : runModelProd.lua
  10 : Utils
  11 : Untitled.ipynb
  12 : README.md
  13 : utilsBatch.lua
  14 : testngrams.lua
  15 : .ipynb_checkpoints
}


In [4]:
dofile("Code/Utils/load_cnn.lua")

cannot open Code/Utils/load_cnn.lua: No such file or directory
stack traceback:
	/home/francisco/torch/install/share/lua/5.2/itorch/main.lua:167: in function </home/francisco/torch/install/share/lua/5.2/itorch/main.lua:160>
	[C]: in function 'dofile'
	[string "local f = function() return dofile("Code/Util..."]:1: in function 'f'
	[string "local f = function() return dofile("Code/Util..."]:1: in main chunk
	[C]: in function 'xpcall'
	/home/francisco/torch/install/share/lua/5.2/itorch/main.lua:210: in function </home/francisco/torch/install/share/lua/5.2/itorch/main.lua:174>
	(...tail calls...)
	/home/francisco/torch/install/share/lua/5.2/lzmq/poller.lua:75: in function 'poll'
	...francisco/torch/install/share/lua/5.2/lzmq/impl/loop.lua:307: in function 'poll'
	...francisco/torch/install/share/lua/5.2/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...francisco/torch/install/share/lua/5.2/lzmq/impl/loop.lua:370: in function 'start'
	/home/francisco/torch/install/share/lua/5.2/itorch/main